# Kominfo

In [ ]:
import requests
from bs4 import BeautifulSoup
import csv
import random

def filter_ads(soup, ad_classes):
    ad_divs = soup.find_all(class_= ad_classes) 
    for ad_div in ad_divs:
        ad_div.extract()

def filter_text(text):
    unwanted_texts = ['Penjelasan :','KATEGORI: HOAKS', 'Link counter:', 'Kategori: Hoaks', 'Link Counter:']

    filtered_text = text
    for unwanted_text in unwanted_texts:
        filtered_text = filtered_text.replace(unwanted_text, '')

    return filtered_text

def extract_titles_and_urls(soup):
    return soup.find_all('a', class_='title')

def extract_title(entry):
    title_text = entry.text.strip()
    if '[HOAKS]' in title_text:
        title_text = title_text.replace('[HOAKS]', '').strip()
    return title_text

def extract_url(entry):
    return "https://www.kominfo.go.id" + entry['href']
    
def extract_text(url):
    news_page = requests.get(url)
    news_soup = BeautifulSoup(news_page.content, 'html.parser')
    
    list_texts = news_soup.find_all('ul')
    for list_text in list_texts:
        list_text.extract()
    
    text_content = news_soup.find('div', class_='typography-block')
    filter_ads(news_soup,'O0')

    if text_content:
        text1 = text_content.text.strip()
        text = filter_text(text1)
        return text
    else:
        return "News content not found for: " + url

train_csv_file = open('training_data_kominfo.csv', 'w', newline='', encoding='utf-8')
test_csv_file = open('Fake.csv', 'w', newline='', encoding='utf-8')

train_csv_writer = csv.writer(train_csv_file)
test_csv_writer = csv.writer(test_csv_file)

train_csv_writer.writerow(['Title', 'URL', 'Text', 'Label'])
test_csv_writer.writerow(['Title', 'URL', 'Text'])

num_pages = 27
for page in range(1, num_pages + 1):
    url = f"https://www.kominfo.go.id/search?search=hoax&page={page}"
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')

    entries = extract_titles_and_urls(soup)
    random.shuffle(entries)

    # Split the data into training (80%) and testing (20%) datasets
    num_train = int(0.8 * len(entries))
    train_entries = entries[:num_train]
    test_entries = entries[num_train:]

    # Keywords
    keywords = ["anies", "prabowo", "presiden", "cawapres", "pemilu", "ganjar", "caslon", "gibran", "tps", "mahfud", "muhaimin", "cak imin", "kpu", "pilpres", "amin"]  # Add more keywords as needed

    # training
    for entry in train_entries:
        title = extract_title(entry)
        url = extract_url(entry)
        keyword_found = False
        for keyword in keywords:
            if keyword in title.lower() and keyword in url.lower():
                keyword_found = True
                break
        if keyword_found:
            text = extract_text(url)
            train_csv_writer.writerow([title, url, text, 0]) 
        else:
            continue

    # test
    for entry in test_entries:
        title = extract_title(entry)
        url = extract_url(entry)
        keyword_found = False
        for keyword in keywords:
            if keyword in title.lower() and keyword in url.lower():
                keyword_found = True
                break
        if keyword_found:
            text = extract_text(url)
            test_csv_writer.writerow([title, url, text])
        else:
            continue

train_csv_file.close()
test_csv_file.close()

In [34]:
import requests
from bs4 import BeautifulSoup
import csv
import datetime
def filter_ads(soup, ad_classes):
    ad_divs = soup.find_all(class_= ad_classes) 
    for ad_div in ad_divs:
        ad_div.extract()

def filter_text(text):
    # Filter unwanted text
    unwanted_texts = ['Jakarta (ANTARA/JACX) – ', 'Jakarta (ANTARA) -']
    filtered_text = text
    for unwanted_text in unwanted_texts:
        filtered_text = filtered_text.replace(unwanted_text, '')
    return filtered_text

def extract_titles_and_urls(soup):
    # Extract titles and URLs from the webpage
    return soup.find_all(class_='post_title post_title_medium')

def extract_title(entry):
    # Extract title text from entry
    return entry.text.strip()

def extract_url(entry):
    # Extract URL from entry
    url_tag = entry.find('a')
    if url_tag:
        return url_tag.get('href')
    else:
        return ""

def extract_complete_date(date_element):
    day = date_element.find('span').text.strip()
    month_year = date_element.text.strip().split()[-1]
    return f"{day}-{month_year}"  

target_dates = "30-04-2024"

url = f"https://www.kominfo.go.id/search?search=hoax&page=1"
page = requests.get(url)
soup = BeautifulSoup(page.content, 'html.parser')

entries = extract_titles_and_urls(soup)

filter_ads(soup, 'col-md-3 wow fadeInRight animated animated')
filter_ads(soup, 'clearfix')

date_elements = soup.find_all(class_='date')
# print(date_elements)

for date_element in date_elements:
    complete_date = extract_complete_date(date_element)
    if complete_date == target_dates:
        print(complete_date)


30-04-2024
30-04-2024
30-04-2024
